In [65]:
import numpy as np
import scipy

In [72]:
input_data = np.genfromtxt('degenerate.csv', delimiter=',')
m, n = input_data.shape[0] - 2, input_data[1] - 1
z = input_data[0, :-1]
c = input_data[1, :-1]
b = input_data[2:, -1]
A = input_data[2:, :-1]
print("Initial Feasible Point (z):", z)
print("Cost Vector (c):", c)
print("Constraint Vector (b):", b)
print("Matrix A:")
print(A)

print(A.shape, b.shape, c.shape, z.shape)
if np.any(A @ z > b):
    print("z does not belong to the solution set")

Initial Feasible Point (z): [3. 2.]
Cost Vector (c): [2. 4.]
Constraint Vector (b): [ 9.  18.   7.   6.   0.   0.   4.5]
Matrix A:
[[ 1.  1.]
 [ 3.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]]
(7, 2) (7,) (2,) (2,)


In [73]:
def find_vertex(A, b, c, z):
    while True:
        tight_rows = np.isclose(A @ z, b)
        num_tight_rows = np.count_nonzero(tight_rows)
        if num_tight_rows == 0:
            u = np.ones_like(z)
        elif num_tight_rows >= A.shape[1]:
            break
        else:
            A1 = A[tight_rows]
            null_space = scipy.linalg.null_space(A1).T
            for vec in null_space:
                if not np.isclose(A @ vec, 0).all():
                    u = vec
                    break
            
            if (A @ (z + 1e6 * u) <= b).all():
                u *= -1
                if (A @ (z + 1e6 * u) <= b).all():
                    print("Cannot find vertex")
                    exit(1)
            
        low = 0
        high = 1e2
        alpha = 0
        print("a")
        while high - low > 1e-12:
            alpha = (low + high) / 2
            z_new = z - (alpha * u)
            if np.any(A @ z_new > b):
                high = alpha
            else:
                low = alpha
            alpha = low
        print("b")
        z = z - (alpha * u)
    return z

def find_optimal_vertex(A, b, c, z):
    while True:
        print("Now at vertex: {}".format(z))
        print("Value of objective function at this vertex = {}".format(np.dot(z, c)))
        tight_rows = np.isclose(A @ z, b)

        if np.count_nonzero(tight_rows) > A.shape[1]:
            print("making system non-degenerate")
            b += np.random.uniform(1e-6, 1e-5, size=b.size)
            z = find_vertex(A, b, c, z)
            continue

        A1 = A[tight_rows]
        A1_inv = np.linalg.inv(A1)
        alphas = A1_inv.T @ c
        if np.all(alphas > 0):
            break
        negative_index = np.where(alphas < 0)[0][0]
        column = A1_inv[:, negative_index]
        if np.all(A @ (z - 1e6 * column) <= b):
            return "Unbounded case"
        low = 0
        high = 1e2
        beta = 0
        while high - low > 1e-9:
            beta = (low + high) / 2
            z_new = z - (beta * column)
            if np.any(A @ z_new > b):
                high = beta
            else:
                low = beta
            beta = low
        z = z - (beta * column)
    return z

vertex = find_vertex(A, b, c, z)
print("Initial vertex: {}".format(vertex))
optimal = find_optimal_vertex(A, b, c, vertex)
print(optimal)

a
b
a
b
Initial vertex: [5.96855898e-13 3.97903932e-13]
Now at vertex: [5.96855898e-13 3.97903932e-13]
Value of objective function at this vertex = 2.7853275241795927e-12
Now at vertex: [4.50000000e+00 3.97903932e-13]
Value of objective function at this vertex = 8.99999999965354
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at vertex: [4.5 4.5]
Value of objective function at this vertex = 26.999999998955047
making system non-degenerate
Now at verte